In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
import random 

     |████████████████████████████████| 63 kB 631 kB/s 
     |████████████████████████████████| 6.4 MB 9.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 1.8 MB/s 


In [14]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols

def get_crypto_syms():
   #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
   'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest


def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row

def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
                data[i-2][j] - data[i-3][j],
                data[i-3][j] - data[i-4][j],
                data[i-4][j] - data[i-5][j],
                data[i-5][j] - data[i-6][j],
                data[i-6][j] - data[i-7][j],
                data[i-7][j] - data[i-8][j],
                data[i-8][j] - data[i-9][j],
                data[i-9][j] - data[i-10][j],
                data[i-10][j] - data[i-11][j],
                data[i-11][j] - data[i-12][j] 
                 ])
           
           
        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = 0
        if data[i][3] > data[i][0]:
            sugg = 1
        
        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)

    grow = []
    srow = []
    llst = []
    data = []
    arr = []

    return np.array(row)
         

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
           data = yf.download(symbol,period="730d",interval="1h",progress=False)
           if data.empty :
              pass
           else:
               if data.shape[0] > 12:
                   unattached_dfs.append(pd.DataFrame(process(data),columns=clmns))
    symbols = []
    symbol = []
    data = []

    data = pd.concat(unattached_dfs)
    data = data.astype(float)
    data = data.astype({'suggestion':'int'})

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,1000)}.csv")
    return xTrain, xTest, yTrain, yTest,data

#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 10
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays:
   get_data(symbols)
    
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data

(1093154, 55) (1093154,)
(121462, 55) (121462,)
(1214616, 56)

1 Failed download:
- EOS-AUD: 1h data not available for startTime=1587491312 and endTime=1650563312. The requested range must be within the last 730 days.
(891369, 55) (891369,)
(99041, 55) (99041,)
(990410, 56)
(1623987, 55) (1623987,)
(180443, 55) (180443,)
(1804430, 56)


In [8]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 2000)              112000    
                                                                 
 dense_10 (Dense)            (None, 1000)              2001000   
                                                                 
 dense_11 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_12 (Dense)            (None, 500)               500500    
                                                                 
 dense_13 (Dense)            (None, 1)                 501       
                                                                 
Total params: 3,615,001
Trainable params: 3,615,001
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(xTrain,yTrain,epochs=50,batch_size=8192,validation_data=(xTest,yTest))

Epoch 1/50
73/92 [======================>.......] - ETA: 55s - loss: 0.7552 - accuracy: 0.5166

KeyboardInterrupt: ignored

In [27]:
model.save("fffirst_model.h5")
model.fit(xTrain,yTrain,epochs=10,batch_size=256,validation_data=(xTest,yTest))
model.save("sssecond_model.h5")

Epoch 1/10
157/157 [==============================] - 1s 7ms/step - loss: 0.1104 - accuracy: 0.9584 - val_loss: 0.2464 - val_accuracy: 0.9365
Epoch 2/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0720 - accuracy: 0.9723 - val_loss: 0.2361 - val_accuracy: 0.9450
Epoch 3/10
157/157 [==============================] - 1s 8ms/step - loss: 0.0601 - accuracy: 0.9763 - val_loss: 0.2315 - val_accuracy: 0.9509
Epoch 4/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0573 - accuracy: 0.9775 - val_loss: 0.2374 - val_accuracy: 0.9570
Epoch 5/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0430 - accuracy: 0.9829 - val_loss: 0.2406 - val_accuracy: 0.9552
Epoch 6/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0443 - accuracy: 0.9826 - val_loss: 0.2465 - val_accuracy: 0.9550
Epoch 7/10
157/157 [==============================] - 1s 7ms/step - loss: 0.0415 - accuracy: 0.9828 - val_loss: 0.2378 - val_accuracy: 0.9559
Epoch 

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
